## Vector Geometry

This is how we described the 3D geometry we were using 40 years ago. I've been looking for a better
description of what this really is and the best I can find is linear algebra applied to 3D geometry. In the
last section we were initially trying to find the target position relative to the camera; then mapping
that back to the target position relative to the robot; and then mapping that back to the robot
position relative to the field.

This required a lot of special math derived for the specific geometry of the current robot with the current
camera mounting. This is really complicated; hard to wrap your head around; and if things change in the
mounting of the camera on the robot, then we may need to re-derive the equations.

We want a more generalized and simple way to describe this system of frames of reference and mapping
between them that is simple to customize for any camera mounting, or for multiple cameras. And this is where
vector geometry comes in.

### The Elements of Vector Geometry

There are several elements of vector geometry that we will talk about:
* **local coordinate systems** - coordinate systems where the origin and axes are generally *attached* to
  an object (like the camera, the robot, the game field), and if you move the object, the coordinate system
  move with it and all the bits associated with the object move together and stay in the same configuration
  relative to the coordinate to the coordinate system;
* **point** - a position expressed in term of a coordinate system;
* **vector** - expresses moving from one point to another point in terms of direction and distance;
* **plane** - a surface that divides space into an *inside* and an *outside*;
* **transformation** - a mapping from the coordinate system of one object (the robot) to another
  coordinate system (the field).

### Point

A point is simply the position of a point in space specified by the X, Y, and Z position relative to the
coordinate system (2D graphing extended to 3D). To prepresent it we need a class that contains the
X, Y, and Z coordinates.

In [3]:
class Point3d 
{
    double m_x = 0.0;
    double m_y = 0.0;
    double m_z = 0.0;
    
    public Point3d()
    {
    
    }
    
    public Point3d(double x, double y, double z)
    {
        m_x = x;
        m_y = y;
        m_z = z;
    }
    
    public double getX()
    {
        return m_x;
    }
    
    public double getY()
    {
        return m_y;
    }
    
    public double getZ()
    {
        return m_z;
    }
    
    /**
     */
    public double distance(Point3d pt)
    {
        double dx = m_x - pt.getX();
        double dy = m_y - pt.getY();
        double dz = m_z - pt.getZ();
        return Math.sqrt((dx * dx) + (dy * dy) + (dz * dz));
    }
}

In [4]:
// a point at the origin
Point3d pt1 = new Point3d();
// a point at some position
Point3d pt2 = new Point3d(-3.0,3.0,5.0);
// get the distance between the points
pt1.distance(pt2);

6.557438524302

### Transformation

Now we need to go into linear algebra and matrices. If you have not done any matrix math before, I will try to
keep this simple. In linear algebra we have 1-dimensional arrays, like a point. We have written the point class
with X, Y, Z member variables. But, it could have been written as an array of 3 values with X, Y, and Z indices of
0, 1, and 2 respectively.

This prepares us to talk about a transform. The idea is that there is a transformation matrix that maps from one
coordinate system (i.e. robot coordinates) to another coordinate system (i.e. field coordinates). The transformation
is a 4x4 matrix, and the transformation (`xfm`) from coordinate system to another coordinate system written as:
```
     [ xfm[0][0] xfm[0][1] xfm[0][2] xfm[0][3] ] [ X ]   [ Xt ]
     | xfm[1][0] xfm[1][1] xfm[1][2] xfm[1][3] | | Y | = | Yt |
     | xfm[2][0] xfm[2][1] xfm[2][2] xfm[2][3] | | Z |   | Zt |
     [ xfm[3][0] xfm[3][1] xfm[3][2] xfm[3][3] ] [ 1 ]   [ 1  ]
```

If you have not done matrix math before, the multiplication is that you walk horizontally across each row of the
matrix and multiply the column value by the row value of the point. Confused? - OK lets walk through that:
```
    Xt = (xfm[0][0] * X) + (xfm[0][1] * Y) + (xfm[0][2] * Z) + (xfm[0][3] * 1)
    Yt = (xfm[1][0] * X) + (xfm[1][1] * Y) + (xfm[1][2] * Z) + (xfm[1][3] * 1)
    Zt = (xfm[2][0] * X) + (xfm[2][1] * Y) + (xfm[2][2] * Z) + (xfm[2][3] * 1)
    1  = (xfm[3][0] * X) + (xfm[3][1] * Y) + (xfm[3][2] * Z) + (xfm[3][3] * 1)
```

And why does that all work out? Well, the heading is a rotation around the Z axis; and the position of the
robot on the field, call it `(Tx,Ty,Tz=0)` (for translation in X and Y) results in a transformation of:

```
     [  cos(heading) sin(heading) 0.0   Tx ] [  X  ]   [  Xt ]
     | -sin(heading) cos(heading) 0.0   Ty | |  Y  | = |  Yt |
     |     0.0          0.0       1.0   Tz | |  Z  |   |  Zt |
     [     0.0          0.0       0.0  1.0 ] [ 1.0 ]   [ 1.0 ]
```

And when we walk through the transformation multipltation:
```
    Xt = ((cos(heading) * X)  + (sin(heading) * Y) + 0.0 +  Tx)
    Yt = ((-sin(heading) * X) + (cos(heading) * Y) + 0.0 +  Ty)
    Zt = (      0.0           +      0.0           +  Z  + 0.0)
    1  = (      0.0           +      0.0           + 0.0 + 1.0)
```

Whoa - that was a lot to digest, so let's look a transformation as it relates to us.


In [17]:
class Vector3d {
    double m_i = 1.0;
    double m_j = 0.0;
    double m_k = 0.0;
    
    public Vector3d()
    {
    
    }
    
    public Vector3d(double i, double j, double k)
    {
        m_i = i;
        m_j = j;
        m_k = k;
    }
    
    public Vector3d(Point3d start, Point3d end)
    {
        m_i = end.getX() - start.getX();
        m_j = end.getY() - start.getY();
        m_k = end.getZ() - start.getZ();
    }
    
    public double getI()
    {
        return m_i;
    }
    
    public double getJ()
    {
        return m_j;
    }
    
    public double getK()
    {
        return m_k;
    }
    
    public Vector3d normalize()
    {
        double length = Math.sqrt((m_i * m_i) + (m_j * m_j) + (m_k * m_k));
        m_i /= length;
        m_j /= length;
        m_k /= length;
        return this;
    }
}





### Vector

In [18]:
Vector3d targetDir = new Vector3d(pt1, pt2);
System.out.println("i,j,k = " + targetDir.getI() + ", " + targetDir.getJ() + ", " + targetDir.getK());
targetDir.normalize();
System.out.println("i,j,k = " + targetDir.getI() + ", " + targetDir.getJ() + ", " + targetDir.getK());



i,j,k = -3.0, 3.0, 5.0
i,j,k = -0.457495710997814, 0.457495710997814, 0.7624928516630234


### Line

### Plane